In [1]:
%load_ext autoreload
%autoreload 2

# Pair

### Search for some products from which to build pairs and stacks of pairs

In [2]:
import asf_search as asf

asf.constants.INTERNAL.CMR_TIMEOUT = 90

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
# results = asf.product_search('S1_181296_IW1_20220219T125501_VV_10AF-BURST')
reference = results[0]

args = asf.ASFSearchOptions(
    **{"start": '2022-02-10', "end": '2022-07-01'}
)
s = reference.stack(args)

### Create a Pair object from 2 products

In [3]:
pair = asf.Pair(reference, s[2])

print(f"pair.ref.properties['sceneName']: {pair.ref.properties['sceneName']}")
print(f"pair.ref_date: {pair.ref_date}\n")

print(f"pair.sec.properties['sceneName']: {pair.sec.properties['sceneName']}")
print(f"pair.sec_date: {pair.sec_date}\n")

print(f"pair.temporal: {pair.temporal}")
print(f"pair.perpendicular: {pair.perpendicular}")

pair.ref.properties['sceneName']: S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252
pair.ref_date: 2022-02-15

pair.sec.properties['sceneName']: S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E
pair.sec_date: 2022-03-11

pair.temporal: 24 days, 0:00:00
pair.perpendicular: -73


### Check the estimated coherence of the pair

In [ ]:
pair.estimate_s1_mean_coherence()

### Create a new pair with a long temporal baseline

In [ ]:
long_pair = pair = asf.Pair(reference, s[10])
print(f"long_pair.temporal: {long_pair.temporal}")

### Since the temporal baseline is greater than 48, an exception is raised when checking coherence

In [ ]:
long_pair.estimate_s1_mean_coherence()

### Demonstrate pair equivalence

Two pairs are equal if they both share the same reference and secondary dates.

In [ ]:
from copy import deepcopy

a = deepcopy(pair)
b = deepcopy(pair)
c = asf.Pair(reference, s[1])
print(f"a.ref_date: {a.ref_date}, a.sec_date: {a.sec_date}")
print(f"b.ref_date: {b.ref_date}, b.sec_date: {b.sec_date}")
print(f"c.ref_date: {c.ref_date}, c.sec_date: {c.sec_date}")
print(f"a == b: {a == b}")
print(f"a == c: {a == c}")

# Stack

### Create a Stack from the reference scene object instantiated above

Stack accepts ASFSearchOptions, which are used in a stack_from_product search

In [ ]:
args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2022-03-02'}
)

stack = asf.Stack(reference, opts=args)
stack.full_stack

### Remove pairs from the full stack, identifying them by ref and sec dates

Both full_stack and subset_stack contain the same Pair objects, so keeping both stacks does not eat up a ton of extra memory

In [ ]:
import numpy as np
import pandas as pd
import datetime

stack.remove_pairs([
    (datetime.date(2022, 1, 10), datetime.date(2022, 1, 22)), # datetime.date
    (pd.to_datetime("2022-1-10"), pd.to_datetime("2022- 2-3")), # pandas.Timestamp
    ("2022-01-10", "2022-02-15"), # isoformat string
    (np.datetime64("2022-01-10"), np.datetime64("2022-02-27")) # numpy.datetime64
])

stack.remove_list

### Look at subset_stack, after having removed some pairs from full_stack

In [ ]:
stack.subset_stack

### Look at connected_substacks

If subset_stack is disconnected, its component stacks are stored in connected_substacks.

Since we have not yet done anything to disconnect the subset_stack, this list contains only one item.

When we use the Network class to make SBAS stacks, we may have multiple connected substacks.



In [ ]:
stack.connected_substacks

## Network

Create a Network from the same reference scene. 

Attempts to create a connected, seasonal SBAS stack based on:
- georeference scene
- season
- start date
- end date
- perpendicular baseline
- temporal baseline
  

In [5]:
import pandas as pd
from datetime import datetime

def get_julian_season(self) -> tuple[int,int]:
    season_start_ts = pd.Timestamp(
        datetime.strptime(f"{season[0]}-0001", "%m-%d-%Y"), tz="UTC"
        )
    season_start_day = season_start_ts.timetuple().tm_yday
    season_end_ts = pd.Timestamp(
        datetime.strptime(f"{season[1]}-0001", "%m-%d-%Y"), tz="UTC"
    )
    season_end_day = season_end_ts.timetuple().tm_yday
    return (season_start_day, season_end_day)

season = ("1-1", "6-25")

opts = asf.ASFSearchOptions(
    **{
        "start": '2014-01-01', 
        "end": '2025-10-02', 
        "season": get_julian_season(season)
        }
)

network = asf.Network(
    geo_reference = reference,
    # multiburst_dict = {1:1},
    perp_baseline=200, 
    inseason_temporal_baseline=12,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

print(f"len(network.full_stack): {len(network.full_stack)}")
print(f"len(network.remove_list): {len(network.remove_list)}")
print(f"len(network.subset_stack): {len(network.subset_stack)}")
print(f"len(network.connected_substacks): {len(network.connected_substacks)}")

len(network.full_stack): 3195
len(network.remove_list): 3033
len(network.subset_stack): 162
len(network.connected_substacks): 4


In [ ]:
network.plot()

In [ ]:
network.plot(network.full_stack)

### Create an updated network with shorter baselines

In [ ]:
network = asf.Network(
    reference,
    perp_baseline=50, 
    inseason_temporal_baseline=12,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

network.plot()

### Create an updated network with longer baselines

In [ ]:
network = asf.Network(
    reference,
    perp_baseline=200, 
    inseason_temporal_baseline=24,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

network.plot()

### Plot full_stack

You can pass an any member stack (or list of member stacks) to the plot function.

In [ ]:
network.plot(stack_dict=network.full_stack)

### Call `Stack.get_insar_pair_ids()` on the largest connected substack

Provides the data structure you want when ordering GAMMA or single-burst interferograms from HyP3

In [ ]:
network.get_insar_pair_ids()

### `Stack.get_insar_pair_ids()` defaults to using the largest connected substack, but you can pass it any member stack

In [ ]:
print(len(network.get_insar_pair_ids()))
print(len(network.get_insar_pair_ids(network.full_stack)))

### MultiBurst Class

In [109]:
# 3x3 adjactent bursts
# Should pass

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "173_370307": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

In [117]:
[i for j in multiburst_dict.values() for i in j]

['IW1', 'IW2', 'IW3', 'IW1', 'IW2', 'IW3', 'IW1', 'IW2', 'IW3']

In [126]:
[f"{burst}_{swath}" for burst, swaths in multiburst_dict.items() for swath in swaths]

['173_370305_IW1',
 '173_370305_IW2',
 '173_370305_IW3',
 '173_370306_IW1',
 '173_370306_IW2',
 '173_370306_IW3',
 '173_370307_IW1',
 '173_370307_IW2',
 '173_370307_IW3']

In [112]:
multiburst_dict

{'173_370305': ('IW1', 'IW2', 'IW3'),
 '173_370306': ('IW1', 'IW2', 'IW3'),
 '173_370307': ('IW1', 'IW2', 'IW3')}

In [80]:
# 3x3 adjactent bursts, but crosses orbital paths
# should fail

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "100_213509": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

Exception: All bursts must belong to the same orbital path.
multiburst_dict contains burst with the following paths: {'173', '100'}

In [157]:
# Contains disconnected bursts and a hole
# Should fail

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "173_370307": ("IW1", ),
    "173_370308": ( "IW3",),
    "173_370309": ("IW1", "IW2", "IW3"),
    "173_370310": ("IW1", "IW3"),
    "173_370311": ("IW1", "IW2", "IW3")
}

mb = asf.MultiBurst(multiburst_dict)

Exception: Multiburst collections must be comprised of a single connected component and have no holes.
Connected Components: 2, Holes: 1

In [164]:
# Diagonal line of bursts
# Should pass

multiburst_dict = {
    "173_370305": ("IW1",),
    "173_370306": ("IW2",),
    "173_370307": ("IW3",),
}

asf.MultiBurst(multiburst_dict)


In [ ]:
# "Vertical" zigzag line of bursts
# Should pass (but will currently fail when ordering from HyP3)

multiburst_dict = {
    "173_370305": ("IW1",),
    "173_370306": ("IW2",),
    "173_370307": ("IW1"),
}

asf.MultiBurst(multiburst_dict)

In [87]:
# "Horizontal" zigzag line of bursts
# Should pass (but will currently fail when ordering from HyP3)

multiburst_dict = {
    "173_370305": ("IW1", "IW3"),
    "173_370306": ("IW2",)
}

asf.MultiBurst(multiburst_dict)


In [169]:

# Intersects the Antimeridan
# Should fail

multiburst_dict = {
    "001_000664": ("IW1", "IW2", "IW3"),
    "001_000665": ("IW1", "IW2", "IW3"),
    "001_000666": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

Exception: No bursts can intersect the Antimeridian

In [97]:
from pystac_client import Client

catalog = Client.open("https://stac.asf.alaska.edu")

collections = catalog.get_collections()
for col in collections:
    print(col.id)





sentinel-1-global-coherence
glo-30-hand


In [100]:
!curl "https://cmr.earthdata.nasa.gov/search/collections.umm_json?concept_id=C2450786986-ASF"




{"hits":1,"took":9,"items":[{"meta":{"revision-id":2,"deleted":false,"format":"application/vnd.nasa.cmr.umm+json","provider-id":"ASF","has-combine":false,"user-id":"mharbin","has-formats":false,"has-spatial-subsetting":false,"native-id":"Sentinel-1_Burst_Map","has-transforms":false,"has-variables":false,"concept-id":"C2450786986-ASF","revision-date":"2024-10-24T16:24:48.764Z","has-temporal-subsetting":false,"concept-type":"collection"},"umm":{"DataLanguage":"eng","AdditionalAttributes":[{"Name":"ASCENDING_DESCENDING","Description":"Describes whether the satellite travel direction was ascending towards the north pole, or descending towards the south pole.","DataType":"STRING"},{"Name":"BURST_ID_FULL","Description":"Unique burst identification generated by combining the PATH_NUMBER, BURST_ID_RELATIVE, and SUBSWATH_NAME (e.g. 002_002154_IW3).","DataType":"STRING"},{"Name":"BURST_ID_RELATIVE","Description":"Burst ID values assigned by ESA, each value identifies a burst cycle, within each s

In [106]:
!curl "https://cmr.earthdata.nasa.gov/search/granules.umm_json?collection_concept_id=C2450786986-ASF&granule_ur=S1_002_002149_IW3-BURSTMAP&page_size=1"




{"hits":1,"took":46,"items":[{"meta":{"concept-type":"granule","concept-id":"G2452512306-ASF","revision-id":1,"native-id":"S1_002_002149_IW3-BURSTMAP","collection-concept-id":"C2450786986-ASF","provider-id":"ASF","format":"application/vnd.nasa.cmr.umm+json","revision-date":"2022-09-13T19:34:45.598Z"},"umm":{"GranuleUR":"S1_002_002149_IW3-BURSTMAP","ProviderDates":[{"Type":"Insert","Date":"2022-09-13T19:34:45.000Z"},{"Type":"Update","Date":"2022-09-13T19:34:45.000Z"}],"CollectionReference":{"ShortName":"Sentinel-1_Burst_Map","Version":"1"},"MetadataSpecification":{"URL":"https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6","Name":"UMM-G","Version":"1.6.6"},"SpatialExtent":{"HorizontalSpatialDomain":{"Geometry":{"GPolygons":[{"Boundary":{"Points":[{"Latitude":1.207538,"Longitude":-19.833227},{"Latitude":1.281845,"Longitude":-19.473805},{"Latitude":1.353175,"Longitude":-19.128078},{"Latitude":1.557218,"Longitude":-19.170735},{"Latitude":1.481462,"Longitude":-19.515549},{"Latitude":1.402729,

In [166]:
import requests

collection_id = "C2450786986-ASF"
granule_ur = "S1_001_000664_IW2-BURSTMAP"

response = requests.get(
    "https://cmr.earthdata.nasa.gov/search/granules.umm_json",
    params={
        "collection_concept_id": collection_id,
        "granule_ur": granule_ur,
        "page_size": 1
    }
)

response.raise_for_status()  # Raises error if response isn't 200 OK

data = response.json()

print(data)


{'hits': 1, 'took': 29, 'items': [{'meta': {'concept-type': 'granule', 'concept-id': 'G2452525133-ASF', 'revision-id': 1, 'native-id': 'S1_001_000664_IW2-BURSTMAP', 'collection-concept-id': 'C2450786986-ASF', 'provider-id': 'ASF', 'format': 'application/vnd.nasa.cmr.umm+json', 'revision-date': '2022-09-13T19:37:16.773Z'}, 'umm': {'GranuleUR': 'S1_001_000664_IW2-BURSTMAP', 'ProviderDates': [{'Type': 'Insert', 'Date': '2022-09-13T19:37:16.000Z'}, {'Type': 'Update', 'Date': '2022-09-13T19:37:16.000Z'}], 'CollectionReference': {'ShortName': 'Sentinel-1_Burst_Map', 'Version': '1'}, 'MetadataSpecification': {'URL': 'https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6', 'Name': 'UMM-G', 'Version': '1.6.6'}, 'SpatialExtent': {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Latitude': 68.51138648783527, 'Longitude': 180.0}, {'Latitude': 68.51274055069817, 'Longitude': 180.0}, {'Latitude': 68.511441, 'Longitude': 179.999275}, {'Latitude': 68.51138648783527, 'Longi

In [156]:
for point in data['items'][0]['umm']['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]['Boundary']['Points']:
    print(point['Longitude'])

180.0
180.0
179.999275
180.0


In [ ]:
https://search.earthdata.nasa.gov/search/granules?p=C2450786986-ASF&pg[0][v]=f&pg[0][gsk]=-start_date&q=C2450786986-ASF&sb[0]=180%2C-33.1627%2C180%2C80.27418&tl=1653868800!1!!&lat=23.55574&long=180&zoom=2.9491955016352556